In [74]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage,SystemMessage,AIMessage,trim_messages

In [75]:
model=ChatOllama(model="gemma3:4b")

In [76]:
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","you are a helpful assistant.Answer all the question "),
        MessagesPlaceholder(variable_name="messages")
    ]
)

In [77]:
parser=StrOutputParser()

In [78]:
chain=prompt|model|parser

In [79]:
result=chain.invoke({"messages":[HumanMessage(content="Is quantum computing only a hype?")]})

In [80]:
print(result)

That's a fantastic and frequently asked question! The short answer is: **it's a complex situation – quantum computing is *partially* hype, but it’s also a rapidly developing field with genuine potential.** Let's break down why:

**Here's where the "hype" comes in:**

* **Overblown Expectations:** For a while, there’s been a lot of media attention and investment driven by the *idea* of quantum computers solving all sorts of problems instantly – like breaking encryption, designing new drugs, or optimizing logistics. This has led to inflated expectations about timelines and capabilities.
* **"Quantum Supremacy" Misinterpretation:** When Google claimed “quantum supremacy” in 2019 (a demonstration that a quantum computer could perform a specific calculation much faster than a classical computer), it was initially seen as a huge leap. However, the task was very contrived and not immediately useful, and it quickly became clear that it was more of a proof of concept than a practical breakthrou

In [81]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [82]:
store={}
def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()

    return store[session_id]


with_message_history=RunnableWithMessageHistory(chain,get_session_history,input_messages_key="messages")

In [83]:
config={"configurable":{"session_id":"chat3"}}
respose=with_message_history.invoke({"messages":[
    HumanMessage("Hello I am X.")
]},config=config)
respose

"Hello X! It's nice to meet you. How can I help you today? 😊 \n\nDo you want to chat, need some information, or perhaps want to play a game? Let me know!"

In [84]:
respose=with_message_history.invoke({"messages":[
    HumanMessage("Where is my name?")
]},config=config)
respose

'That’s a clever question! Your name is X. 😊 \n\nI was just asking you what *your* name was. 😉 \n\nIs there anything else you’d like to talk about?'

### Trimmer

In [90]:
trimmer=trim_messages(
    max_tokens=30,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False
)


messages=[
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi I am X"),
    AIMessage(content="Hi")
]


trimmer.invoke(messages)

/home/workstation-p/anaconda3/envs/conda-env-3-12/lib/python3.12/site-packages/langchain_core/language_models/base.py:336: UserWarning: Using fallback GPT-2 tokenizer for token counting. Token counts may be inaccurate for non-GPT-2 models. For accurate counts, use a model-specific method if available.
  return len(self.get_token_ids(text))


[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='hi I am X', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hi', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[])]

In [91]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough
chain=(RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)|prompt|model)

In [94]:
response=chain.invoke({"messages":messages+[HumanMessage("What is my name?")]})

In [95]:
response.content

'Your name is X! 😊 \n\nIs there anything I can help you with today, X?'

In [97]:
with_message_history=RunnableWithMessageHistory(
    chain,get_session_history,input_messages_key="messages"
)
config={"configurable":{"session_id":"chat5"}}
response=with_message_history.invoke({"messages":HumanMessage("What is my name?")},config=config)

In [98]:
response.content

"As an AI, I have no way of knowing your name! You haven't told me. 😊 \n\nTo help me respond to you better in the future, could you tell me your name?"

In [100]:
config={"configurable":{"session_id":"chat3"}}
response=with_message_history.invoke({"messages":HumanMessage("What is my name?")},config=config)
response.content

"As an AI, I don’t know your name! You haven't told me. 😊 \n\nWhat *is* your name?"